# Lab 6 - Recurrent Neural Networks

## Problems: (Geron, 15.10)

<img width = 800 src = http://www.bach-chorales.com/Images/ChoraleImages/Image_BWV_0133_6.jpg>

Download the [Bach chorales dataset](https://homl.info/bach) and unzip it. It is composed of 382 chorales composed by Johann Sebastian Bach. Each chorale is 100 to 640 time steps long, and each time step contains 4 integers, where each integer corresponds to a note’s index on a piano (except for the value 0, which means that no note is played).

Train a model — recurrent, convolutional, or both — that can predict the next time step (four notes), given a sequence of time steps from a chorale. Then use this model to generate Bach-like music, one note at a time: you can do this by giving the model the start of a chorale and asking it to predict the next time step, then appending these time steps to the input sequence and asking the model for the next note, and so on. Also make sure to check out [Google’s Coconet model](https://magenta.tensorflow.org/coconet), which was used for a nice Google doodle about Bach.

#### Further information

At each time step, each chorales has 4 notes. Looking at the CSV for one of  the files we see that the columns are __note0__, __note1__, __note2__ and __note3__, with each row corresponding to a timestep. Each of the numbers corresponds to a piano key. 

Python can construct audio and Jupyter can play it in a Jupyter widget:

In [ ]:
from IPython.display import Audio
import numpy as np
import pandas as pd

signal = np.random.random(750)
Audio(signal, rate=250)

The audio is then constructed by a simple sum of sine functions. I've uploaded in index between the note position and the keys taken from Wikipeidia (https://en.wikipedia.org/wiki/Piano_key_frequencies).

In [ ]:
piano_path = "https://raw.githubusercontent.com/tipthederiver/Math-7243-2020/master/Labs/Lab%206/Piano%20Notes.csv"
notes = pd.read_csv(piano_path, encoding= 'unicode_escape')
notes

We can then get the frequency from the key position by

In [ ]:
notes[notes["Key Position"]==1]

The function below will take a sequence of notes and turn it into audio.

In [ ]:
def gen_audio(song, notes, framerate=22050, L=.25):
    framerate = 22050  ## Standard Framerate
    N = len(song) # Number of Notes
    L = .25        # Note Length in Seconds
    W = int(framerate*L) # Window Size
    t = np.linspace(0,L,W)
    data = np.zeros(W*N)

    for i in range(N): 
        F = notes["Frequency (Hz)"].iloc[song[i]+1]
        data[W*i:W*(i+1)] = np.sin(2*np.pi*F*t)
        
    return data

 I've included two note snippets below. These are __note0__ and __note1__ from chorale 305:

In [ ]:
song_0 = [65,65,65,65,72,72,70,70,69,69,67,67,65,65,65,65,72,72,72,72,74,74,74,74,74,74,74]
song_1 = [60,60,60,60,60,60,60,60,60,60,60,60,62,62,64,64,65,65,65,65,65,65,65,65,65,65,65]

You can generate the tunes individually:

In [ ]:
data_0 = gen_audio(song_0, notes)
Audio(data,rate=22050)

or to hear them together, simply add the sine-wave representation:

In [ ]:
data_0 = gen_audio(song_0, notes)
data_1 = gen_audio(song_1, notes)
Audio(data_0 + data_1,rate=22050)

#### Rough Outline: 

This entire project can be completed without ever listening to the audio files and instead just treating the sequences as sequences. Your pattern should roughly be the text generator above, although now at each time step we have 4 notes, not a single letter. A rough outline is as follows:

* Load a single chorale using Panda's `read_csv` function.
* Construct the training data as we did for the text generator: the `X_train` will be sequences of $K$ timesteps, each time step containing the 4 notes. It is your choice if you leave them as integers of one-hot encode the notes. 
* The labels `y_train` will be sequences of length $K$ timesteps shifted by 1.
* Construct a simple RNN model with 64 LSTM nodes. Your input and output shape will be $K\times 4$ if you do not one-hot encode and $K\times 4\times 108$ if you do one-hot encode.
* After your get your network running on one chorale, expand your dataset by adding sequences from other chorales. 